In [ ]:
notebookName = 'audio-deepfake-detection-testing'
runJobId = 'ASVspoof-2019_small-eval-1'
# runJobId = 'ASVspoof-2019_training'     # This should fail

import joblib
import numpy as np
from tensorflow.keras.utils import to_categorical

import configuration.configuration as configuration
from mel_spectrogram.mel_spectrogram import MelSpectrogramGenerator
from notebook_utils import notebookToPython
from processors.basic_model_evaluation_processor import BasicModelEvaluationProcessor
from readers.label_reader import readTrainingLabelsWithJob

In [ ]:
config = configuration.ConfigLoader('config.yml')

notebookToPython(notebookName)
job = config.getJobConfig(runJobId)

if (job.newModelGenerated):
    raise ValueError("This notebook is meant for testing. Select a job with a value for 'persisted-model' set.")

Write python file
Generating new model name: ASVspoof-2019_training_2025-03-20T20-34-43.149507.libjob
Assigned model name: ASVspoof-2019_training_2025-03-20T20-34-43.149507.libjob


ValueError: This notebook is meant for testing. Select a job with a value for 'persisted-mode' set.

In [3]:
generator = MelSpectrogramGenerator()
model = joblib.load(job.persistedModel)
evaluationProc = BasicModelEvaluationProcessor(job, model)

In [4]:
fullDataPath = job.fullJoinFilePath(job.dataPathRoot, job.dataPathSuffix)
labels = readTrainingLabelsWithJob(job)

Loading C:/Users/tubas/workspace/Deepfake/data/ASVspoof-2019/LA/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.eval-small.trl.txt...


In [5]:
for filename, label in labels.items():
    print(f"\nEvaluating {filename} - {label}")
    X, y = generator.generateMelSpectrogram(job, fullDataPath, filename, label)
    evaluationProc.process(X, y)

print("\n")
evaluationProc.reportSnapshot()


Evaluating LA_E_5932896 - 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
  Processed: 1 - Score: 1.0

Evaluating LA_E_5849185 - 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
  Processed: 1 - Score: 0.0

Evaluating LA_E_6163791 - 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
  Processed: 1 - Score: 1.0

Evaluating LA_E_4581379 - 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
  Processed: 1 - Score: 1.0


timestamp: 2025-03-21T01:30:25.239948+00:00
model file: ASVspoof-2019-1_2025-03-19T22-03-42.178503.libjob
test file count: 4
accuracy_score: 0.75

job: {
    "jobId": "ASVspoof-2019_small-eval-1",
    "dataPathRootRaw": "$HOMEDRIVE$HOMEPATH/workspace/Deepfake/data/ASVspoof-2019",
    "dataPathRoot": "C:/Users/tubas/workspace/Deepfake/data/ASVspoof-2019",
    "dataPathSuffix": "LA/ASVspoof2019_LA_eval_small/flac",
    "dataExtension": ".flac",
    "labelFilename": "LA/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.eval-small.trl.txt",
    "executeToCategoricalForLabels": true,
    "numClasses": 2,
    "sampleRa